In [2]:
import torch
from torch import nn
import matplotlib.pyplot as plt
torch.__version__

'2.5.1+cu124'

In [3]:
weight=0.7
bias=0.3
start=0
end=1
step=0.02
X= torch.arange(start,end,step).unsqueeze(dim=1) # adds extra dimension(square bracket) for models later on
y= weight*X+bias
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [4]:
len(X),len(y)

(50, 50)

In [5]:
train_split=int(0.8*len(X))
X_train,y_train=X[:train_split],y[:train_split]
X_test,y_test=X[train_split:],y[train_split:]
len(X_train),len(y_train),len(X_test),len(y_test)


(40, 40, 10, 10)

In [6]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights=nn.Parameter(torch.randn(1,
                                          requires_grad=True,
                                          dtype=torch.float))
    self.bias=nn.Parameter(torch.randn(1,
                                        requires_grad=True,
                                        dtype=torch.float))
  def forward(self,x: torch.Tensor) -> torch.Tensor:
     return self.weights*x +bias




In [7]:
RANDOM_SEED=42
torch.manual_seed(RANDOM_SEED)
model_0=LinearRegression()
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [8]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [9]:
y_preds= model_0(X_test)
y_preds

tensor([[0.5694],
        [0.5761],
        [0.5828],
        [0.5896],
        [0.5963],
        [0.6030],
        [0.6098],
        [0.6165],
        [0.6232],
        [0.6300]], grad_fn=<AddBackward0>)

In [10]:
y_test

tensor([[0.8600],
        [0.8740],
        [0.8880],
        [0.9020],
        [0.9160],
        [0.9300],
        [0.9440],
        [0.9580],
        [0.9720],
        [0.9860]])

In [11]:
loss_fun=nn.L1Loss()
optimizer= torch.optim.SGD(params=model_0.parameters(),
                           lr=0.01
                           )


In [12]:
epochs=1
for epoch in range(epochs):
  model_0.train()
  y_pred=model_0(X_train)
  loss=loss_fun(y_pred,y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

loss


tensor(0.1417, grad_fn=<MeanBackward0>)

In [13]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3406])), ('bias', tensor([0.1288]))])

In [14]:
with torch.inference_mode():
  y_pred_new=model_0(X_test)


In [15]:
loss_fun(y_pred_new,y_test)

tensor(0.3199)

In [16]:
from pathlib import Path
model_path=Path("models")
model_path.mkdir(parents=True,exist_ok=True)
model_name="01_model.pth"
model_save_path=model_path/model_name

torch.save(obj=model_0.state_dict(),
           f=model_save_path)

In [17]:
loaded_model_0=LinearRegression()
loaded_model_0.load_state_dict(torch.load(f=model_save_path))

<ipython-input-17-3355e510ff35>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model_0.load_state_dict(torch.load(f=model_save_path))


<All keys matched successfully>

In [18]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.3406])), ('bias', tensor([0.1288]))])

In [19]:
loaded_model_0.eval()
with torch.inference_mode():
  loaded_model_preds=loaded_model_0(X_test)
loaded_model_preds

tensor([[0.5725],
        [0.5793],
        [0.5861],
        [0.5929],
        [0.5997],
        [0.6065],
        [0.6133],
        [0.6202],
        [0.6270],
        [0.6338]])

In [20]:
import torch
from torch import nn
import matplotlib.pyplot as plt
torch.__version__

'2.5.1+cu124'

In [21]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [22]:
weight=0.1
bias=0.2
start=0
end=1
step=0.02
X=torch.arange(start,end,step).unsqueeze(dim=1)
y=X*weight+bias


In [23]:
train_split=int(0.7*len(X))
X_train,y_train=X[:train_split],y[:train_split]
X_test,y_test=X[train_split:],y[train_split:]

In [33]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_layer=nn.Linear(in_features=1,
                                out_features=1)
  def forward(self,x:torch.Tensor) -> torch.Tensor:
      return self.linear_layer(x)

torch.manual_seed(42)
model_1=LinearRegression()
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
             ('linear_layer.bias', tensor([0.8300]))])

In [34]:
model_1.to(device)
next(model_1.parameters()).device


device(type='cuda', index=0)

In [35]:
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7645]], device='cuda:0')),
             ('linear_layer.bias', tensor([0.8300], device='cuda:0'))])

In [36]:
loss_fn=nn.L1Loss()
optimizer=torch.optim.SGD(params=model_1.parameters(),
                          lr=0.01
                          )

In [37]:
X_train=X_train.to(device)
y_train=y_train.to(device)
X_test=X_test.to(device)
y_test=y_test.to(device)

epochs=200
for epoch in range(epochs):
  model_1.train()
  y_pred=model_1(X_train)
  loss=loss_fn(y_pred,y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  model_1.eval()
  with torch.inference_mode():
    test_pred=model_1(X_test)
    test_loss = loss_fn(test_pred,y_test)
  if epoch%10==0:
    print(loss,test_loss)

tensor(0.8560, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.1754, device='cuda:0')
tensor(0.7444, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.0468, device='cuda:0')
tensor(0.6328, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.9182, device='cuda:0')
tensor(0.5213, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.7897, device='cuda:0')
tensor(0.4097, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.6611, device='cuda:0')
tensor(0.2982, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5326, device='cuda:0')
tensor(0.1866, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4040, device='cuda:0')
tensor(0.1012, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.2938, device='cuda:0')
tensor(0.0744, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.2308, device='cuda:0')
tensor(0.0662, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1966, device='cuda:0')
tensor(0.0624, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1768, device='cuda:0')
tensor(0.0595, device

In [38]:
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.2988]], device='cuda:0')),
             ('linear_layer.bias', tensor([0.1283], device='cuda:0'))])

In [39]:
model_1.eval()
with torch.inference_mode():
  y_preds=model_1(X_test)
y_preds

tensor([[0.3375],
        [0.3434],
        [0.3494],
        [0.3554],
        [0.3614],
        [0.3674],
        [0.3733],
        [0.3793],
        [0.3853],
        [0.3913],
        [0.3972],
        [0.4032],
        [0.4092],
        [0.4152],
        [0.4211]], device='cuda:0')